In [3]:
#%pip install opencv-python


     ---------------------------------------- 38.2/38.2 MB 4.8 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\naveen.muralidharan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [9]:
import os
import time
import pickle

import cv2

import numpy as np  

from keras import applications
from keras.applications.inception_v3 import preprocess_input
from keras.utils.image_utils import img_to_array
from keras.models import Model

In [14]:

model_load_path = os.path.abspath('./models')
test_video_path = ''
video_summary_save_path = os.path.abspath('./results_summary')

start_time = time.time()

base_model = applications.inception_v3.InceptionV3(include_top=True, weights='imagenet', input_tensor=None,
                                                   input_shape=None, pooling=None, classes=1000)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

#Project Model- Loading the saved Inception models
loaded_model1 = pickle.load(open(f'{model_load_path}/model1_inception_svm.sav', 'rb'))
loaded_model2 = pickle.load(open(f'{model_load_path}/model2_inception_svm.sav', 'rb'))

#Path for the video to be summarized
vidcap = cv2.VideoCapture(f'{test_video_path}')

count = 0
bufferCount = 0

globalWideVideo = []
globalOutVideo = []
globalSixVideo = []
globalNoBallVideo = []
globalNoActionVideo = []
buffer = []

th = 5
buff_th = 250
globalWideCounter = 0
globalOutCounter = 0
globalSixCounter = 0
globalNoBallCounter = 0
globalNoActionCounter = 0

while (True):
    success, img = vidcap.read()
    
    if success:
        bufferCount = bufferCount + 1
        buffer.append(img)
        height, width, layers = img.shape
        size = (width, height)
        count = count + 1
       # print ('success')
        img1 = cv2.resize(img, (299, 299))
        x = img_to_array(img1)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        #Feature Extraction Step
        features = model.predict(x) #Inception V3 Model
        predicted_values = loaded_model1.predict(features.reshape(1, -1)) 
        if predicted_values==2:
            predicted_values_2 = loaded_model2.predict(features.reshape(1, -1))
            choices = {'1':'noball', '2':'out', '3':'six', '4':'wide', '5':'noaction'}
            result = choices.get(str(int(predicted_values_2)), 'default')
            if result == 'noball':
                globalNoBallCounter = globalNoBallCounter + 1
                print('noball:')
            if result == 'out':
                globalOutCounter = globalOutCounter + 1
                print('out:')
            if result == 'six':
                globalSixCounter = globalSixCounter + 1
                print('six:')
            if result == 'wide':
                globalWideCounter = globalWideCounter + 1
                print('wide:')
            if result == 'noaction':
                globalNoActionCounter = globalNoActionCounter + 1
                print('noaction:')
                
    else:
        break
   
    if bufferCount == buff_th:
        actionCount = {'noball': globalNoBallCounter, 'out': globalOutCounter, 
                       'six': globalSixCounter, 'wide': globalWideCounter}

        verdict = max(actionCount, key=actionCount.get)
        if verdict == 'noball' and globalNoBallCounter >th:
            globalNoBallVideo.append(buffer)
        if verdict == 'out'and globalOutCounter >th:
            globalOutVideo.append(buffer) 
        if verdict == 'six'and globalSixCounter >th:
            globalSixVideo.append(buffer)
        if verdict == 'wide' and globalWideCounter >th:
            globalWideVideo.append(buffer)
       
        bufferCount = 0
        globalWideCounter = 0
        globalOutCounter = 0
        globalSixCounter = 0
        globalNoBallCounter = 0
        globalNoActionCounter = 0
        buffer = []
    
actionCount = {'noball': globalNoBallCounter, 'out': globalOutCounter,
               'six': globalSixCounter, 'wide': globalWideCounter}
winner = max(actionCount, key=actionCount.get)
if winner == 'noball' and globalNoBallCounter >th:
    globalNoBallVideo.append(buffer)
if winner == 'out'and globalOutCounter >th:
    globalOutVideo.append(buffer) 
if winner == 'six'and globalSixCounter >th:
    globalSixVideo.append(buffer)
if winner == 'wide' and globalWideCounter >th:
    globalWideVideo.append(buffer)

cv2.destroyAllWindows()

print ('Summarizing Video...')

if globalNoBallVideo!=[]:
    noBallVideo = cv2.VideoWriter(f'{video_summary_save_path}/no_ball.avi', cv2.VideoWriter_fourcc(*'DIVX'),
                                  25, size)
    for i in range(len(globalNoBallVideo)):
        for j in range(len(globalNoBallVideo[i])):
            # writing to a image array
            noBallVideo.write(globalNoBallVideo[i][j])
    noBallVideo.release()

if globalOutVideo!=[]:
    outVideo = cv2.VideoWriter(f'{video_summary_save_path}/out.avi', cv2.VideoWriter_fourcc(*'DIVX'),
                               25, size)
    for i in range(len(globalOutVideo)):
        for j in range(len(globalOutVideo[i])):
            # writing to a image array
            outVideo.write(globalOutVideo[i][j])
    outVideo.release()    

if globalWideVideo!=[]:
    wideVideo = cv2.VideoWriter(f'{video_summary_save_path}/wide.avi', cv2.VideoWriter_fourcc(*'DIVX'),
                                25, size)
    for i in range(len(globalWideVideo)):
        for j in range(len(globalWideVideo[i])):
            # writing to a image array
            wideVideo.write(globalWideVideo[i][j])
    wideVideo.release()

if globalSixVideo!=[]:
    sixVideo = cv2.VideoWriter(f'{video_summary_save_path}/sixes.avi', cv2.VideoWriter_fourcc(*'DIVX'),
                               25, size)
    for i in range(len(globalSixVideo)):
        for j in range(len(globalSixVideo[i])):
            # writing to a image array
            sixVideo.write(globalSixVideo[i][j])
    sixVideo.release()
    
print('--- %s seconds ---' % (time.time() - start_time))




1/1 [==============================] - 0s 76ms/step
noaction:
1/1 [==============================] - 0s 76ms/step
noaction:
1/1 [==============================] - 0s 75ms/step
noaction:
1/1 [==============================] - 0s 85ms/step
noaction:
1/1 [==============================] - 0s 66ms/step
noaction:
1/1 [==============================] - 0s 69ms/step
noaction:
1/1 [==============================] - 0s 78ms/step
noball:
1/1 [==============================] - 0s 84ms/step
noball:
1/1 [==============================] - 0s 69ms/step
noaction:
1/1 [==============================] - 0s 86ms/step
noaction:
1/1 [==============================] - 0s 73ms/step
noaction:
1/1 [==============================] - 0s 78ms/step
noaction:
1/1 [==============================] - 0s 66ms/step
noaction:
1/1 [==============================] - 0s 68ms/step
noaction:
1/1 [==============================] - 0s 74ms/step
noaction:
1/1 [==============================] - 0s 72ms/step
noaction:
1/1 [=======